In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, FloatType, IntegerType, DateType, MapType
import pyspark.sql.functions as F
import re
import yaml

#define path to your yaml file
yaml_file_path= 'config.yaml'

with open(yaml_file_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

spark = SparkSession \
    .builder \
    .appName("final_project") \
    .config("spark.jars", config['spark']['path']) \
    .getOrCreate()

In [6]:
def extract():
    try:
        # CSV path
        csv = config['csv']['path']
        # Read raw_data
        df = spark.read.csv(csv, header=True, inferSchema=False)
        return df
    except Exception as e:
        raise Exception(f"An error occurred during data extraction: {str(e)}")
        spark.stop()

In [7]:
df=extract()
df.select("Company Profile").show(truncate=False)

+--------------------------------------+
|Company Profile                       |
+--------------------------------------+
|"{""Sector"":""Financial Services""   |
|"{""Sector"":""Insurance""            |
|"{""Sector"":""Telecommunications""   |
|"{""Sector"":""Electronics""          |
|"{""Sector"":""Industrial""           |
|"{""Sector"":""Healthcare""           |
|"{""Sector"":""Retail""               |
|"{""Sector"":""Retail""               |
|"{""Sector"":""Energy""               |
|"{""Sector"":""Automotive Parts""     |
|"{""Sector"":""IT Services""          |
|"{""Sector"":""Financial Technology"" |
|"{""Sector"":""Waste Management""     |
|"{""Sector"":""Financial Services""   |
|"{""Sector"":""Food and Beverage""    |
|"{""Sector"":""Mining""               |
|"{""Sector"":""Healthcare""           |
|"{""Sector"":""Healthcare""           |
|"{""Sector"":""Real Estate/Logistics""|
|"{""Sector"":""Real Estate""          |
+--------------------------------------+
only showing top

In [17]:
def clean():
    try:
        df = extract()
        # Changing -ve company size to positive
        df = df.withColumn("Company Size", F.when(F.col("Company Size") < 0, -F.col("Company Size")).otherwise(F.col("Company Size")))
        
        # List of columns to drop
        dropped = ["Contact Person", "Contact", "Benefits"]
        # Drop columns
        df = df.drop(*dropped)
        
        # Changing to standard data types
        df = df.withColumn("Job Id", df["Job Id"].cast(LongType()))\
               .withColumn("latitude", df["latitude"].cast(FloatType()))\
               .withColumn("longitude", df["longitude"].cast(FloatType()))\
               .withColumn("Company Size", df["Company Size"].cast(IntegerType()))\
               .withColumn("Job Posting Date", df["Job Posting Date"].cast(DateType()))
        
        # sector_pattern = r'"Sector":"([^"]+)"'

        # # Use regexp_extract to extract the value of "Sector" into a new column "Sector"
        # df = df.withColumn("Sector", F.regexp_extract(df["Company Profile"], sector_pattern, 1))
        df = df.withColumn("Sector", F.substring_index(F.col("Company Profile"), '":"', -1))
        df = df.withColumn("Sector", F.regexp_replace(F.col("Sector"), '"', ''))
        df=df.drop("Company Profile")




        return df
    except Exception as e:
        raise Exception(f"An error occurred during data cleaning: {str(e)}")
        spark.stop()

In [18]:
dff=clean()
# dff.select('Sector').show(truncate=False)
dff.show(10)

+----------------+-------------+--------------+------------+------------------+---------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|          Job Id|   experience|qualifications|Salary Range|          location|        country|latitude|longitude|Work Type|Company Size|Job Posting Date|preference|           Job Title|                Role|         Job Portal|     Job Description|              skills|    responsibilities|             company|            Sector|
+----------------+-------------+--------------+------------+------------------+---------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+
| 75055

In [19]:
#udf to calculate avg
def calculate_average(range_str):
    # Use regular expression to extract numbers
    numbers = re.findall(r'\d+', range_str)
    if len(numbers) == 2:
        lower = int(numbers[0])
        upper = int(numbers[1])
        avg = (lower + upper) / 2
        return avg
    else:
        return None


In [20]:
def transform():
    try:
        df=clean()
        calculate_average_udf = F.udf(calculate_average)
        # Add a new column with the calculated average
        new_df = df.withColumn("Average", calculate_average_udf(df["Salary Range"]))
        new_df = new_df.withColumnRenamed("Average","Average Salary")
        new_df = new_df.withColumn("Average Salary",  
                                  new_df["Average Salary"] 
                                  .cast('int')) 

        
        # Changing gender preference from "both" to "Male or Female"
        new_df = new_df.withColumn("Preference", F.when(F.col("Preference") == "Both", "Male or Female").otherwise(F.col("Preference")))
        new_df.show()

        #Dividing companies into tiers according to the company size

        quartiles = new_df.stat.approxQuantile("company size", [0.25, 0.75], 0.0)
        q1, q3 = quartiles
        new_df = new_df.withColumn("CompanyTier",
        F.when(F.col("company size") <= q1, "Tier-1 (Low)")
        .when((F.col("company size") > q1) & (F.col("company size") <= q3), "Tier-2 (Medium)")
        .when(F.col("company size") > q3, "Tier-3 (High)")
        .otherwise("Uncategorized"))


        #Dividing jobs into three categories according to their salary.
        
        quartiles = new_df.stat.approxQuantile("Average Salary", [0.25, 0.75], 0.0)
        q1, q3 = quartiles
        new_df = new_df.withColumn("SalaryLevel",
        F.when(F.col("Average Salary") <= q1, "Low Pay")
        .when((F.col("Average Salary") > q1) & (F.col("Average Salary") <= q3), "Average Pay")
        .when(F.col("Average Salary") > q3, "High Pay")
        .otherwise("Uncategorized"))
        

        # Differentiate qualifications according to their initials

        new_df = new_df.withColumn("QualificationCategory", F.when(F.col("qualifications").startswith("M"), "Masters")
        .when(F.col("qualifications").startswith("B"), "Bachelors")
        .when(F.col("qualifications").startswith("P"), "PhD")
        .otherwise("Uncategorized"))


        return new_df

    except Exception as e:
        raise Exception(f"An error occurred during data transformation: {str(e)}")
        spark.stop()

In [41]:
dff=transform()
dff.printSchema()

+----------------+-------------+--------------+------------+--------------------+---------------+--------+---------+---------+------------+----------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|          Job Id|   experience|qualifications|Salary Range|            location|        country|latitude|longitude|Work Type|Company Size|Job Posting Date|    Preference|           Job Title|                Role|         Job Portal|     Job Description|              skills|    responsibilities|             company|     Company Profile|Average Salary|
+----------------+-------------+--------------+------------+--------------------+---------------+--------+---------+---------+------------+----------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------

ERROR:root:KeyboardInterrupt while sending command.==>              (3 + 1) / 4]
Traceback (most recent call last):
  File "/home/ubuntu/Desktop/final_project/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/Desktop/final_project/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [21]:
def load():
    try:
        dff=transform()
        ##Load the clean data in postgres
        dff.write.format('jdbc').options(url=config['postgres']["url"],driver = config['postgres']["driver"], dbtable = config['postgres']["dbtable"], user=config['postgres']["user"],password=config['postgres']["password"]).mode('overwrite').save()
        return dff
    except Exception as e:
        raise Exception(f"An error occurred during loading the data: {str(e)}")
        spark.stop()    


In [22]:
load()

+----------------+-------------+--------------+------------+--------------------+---------------+--------+---------+---------+------------+----------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|          Job Id|   experience|qualifications|Salary Range|            location|        country|latitude|longitude|Work Type|Company Size|Job Posting Date|    Preference|           Job Title|                Role|         Job Portal|     Job Description|              skills|    responsibilities|             company|              Sector|Average Salary|
+----------------+-------------+--------------+------------+--------------------+---------------+--------+---------+---------+------------+----------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------

DataFrame[Job Id: bigint, experience: string, qualifications: string, Salary Range: string, location: string, country: string, latitude: float, longitude: float, Work Type: string, Company Size: int, Job Posting Date: date, Preference: string, Job Title: string, Role: string, Job Portal: string, Job Description: string, skills: string, responsibilities: string, company: string, Sector: string, Average Salary: int, CompanyTier: string, SalaryLevel: string, QualificationCategory: string]